In [ ]:
!pip install opencv-python-headless ultralytics numpy scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

# Loading the model in the environment
model_path = '/content/best.pt'
model = YOLO(model_path)


In [ ]:
# Video paths (update these based on your Drive)
broadcast_video = '/content/broadcast.mp4'
tactician_video = '/content/tacticam.mp4'


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import collections

# Load model and videos
model = YOLO('/content/best.pt')
cap_broadcast = cv2.VideoCapture('/content/broadcast.mp4')  # Replace with your broadcast video path
cap_tactician = cv2.VideoCapture('/content/tacticam.mp4')  # Replace with your tactician video path

if not (cap_broadcast.isOpened() and cap_tactician.isOpened()):
    raise ValueError("Error opening video files")

# Initialize mapping with counters for voting
player_mapping = {}
vote_counts = collections.defaultdict(lambda: collections.defaultdict(int))
frames_to_process = 30  # Process first 30 frames

for frame_count in range(frames_to_process):
    ret_b, frame_b = cap_broadcast.read()
    ret_t, frame_t = cap_tactician.read()
    if not (ret_b and ret_t):
        break

    results_b = model(frame_b)
    results_t = model(frame_t)

    def get_player_centroids(results):
        centroids = []
        for result in results:
            for box in result.boxes:
                if int(box.cls) == 0:  # Assuming class 0 is player
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    centroid = ((x1 + x2) / 2, (y1 + y2) / 2)
                    centroids.append(centroid)
        return centroids

    centroids_b = get_player_centroids(results_b)
    centroids_t = get_player_centroids(results_t)

    for i, cent_b in enumerate(centroids_b):
        min_dist = float('inf')
        match_idx = -1
        for j, cent_t in enumerate(centroids_t):
            dist = np.sqrt((cent_b[0] - cent_t[0])**2 + (cent_b[1] - cent_t[1])**2)
            if dist < min_dist and dist < 500:  # Increased threshold
                min_dist = dist
                match_idx = j
        if match_idx != -1:
            vote_counts[f'player_{i}'][f'player_{match_idx}'] += 1

# Determine best matches based on votes
for player_b, votes in vote_counts.items():
    best_match = max(votes, key=votes.get) if votes else None
    if best_match and votes[best_match] > frames_to_process * 0.3:  # Require at least 30% agreement
        player_mapping[player_b] = best_match

# Save mapping
with open('/content/player_mapping.txt', 'a') as f:
    for k, v in player_mapping.items():
        f.write(f"{k}: {v}\n")

cap_broadcast.release()
cap_tactician.release()
cv2.destroyAllWindows()

print("Player mapping updated and saved to player_mapping.txt")


0: 384x640 3 players, 66.8ms
Speed: 2.9ms preprocess, 66.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 2 referees, 52.9ms
Speed: 2.9ms preprocess, 52.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 35.2ms
Speed: 2.8ms preprocess, 35.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 1 referee, 40.8ms
Speed: 2.6ms preprocess, 40.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 37.4ms
Speed: 3.9ms preprocess, 37.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 1 referee, 40.4ms
Speed: 3.8ms preprocess, 40.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 35.7ms
Speed: 3.5ms preprocess, 35.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 38.6ms
Speed: 3.6ms preprocess, 38.6ms inference, 

# Re-identification in a Single Feed

In [ ]:
!pip install nest_asyncio

import cv2
import numpy as np
from ultralytics import YOLO
import asyncio
import nest_asyncio
import platform

# Allow

[7]
36s
 nested event loops in Colab
nest_asyncio.apply()

# Load the YOLOv11 model
model_path = '/content/best.pt'
model = YOLO(model_path)

# Video path
video_path = '/content/15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise ValueError("Error opening video file")

# Initialize player tracking
player_ids = {}
frame_count = 0
FPS = 30
INITIAL_FRAMES = 30  # Assign IDs in first second

async def main():
    global frame_count
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        results = model(frame)

        # Extract bounding boxes
        def get_player_boxes(results):
            boxes = []
            for result in results:
                for box in result.boxes:
                    if int(box.cls) == 0:  # Assuming class 0 is player
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        conf = box.conf[0]
                        boxes.append((x1, y1, x2, y2, conf))
            return boxes

        current_boxes = get_player_boxes(results)

        # Assign IDs in initial frames
        if frame_count <= INITIAL_FRAMES and current_boxes:
            for i, box in enumerate(current_boxes):
                centroid = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
                player_ids[centroid] = f'player_{i}'

        # Match re-entered players
        if frame_count > INITIAL_FRAMES and current_boxes:
            for box in current_boxes:
                centroid = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
                min_dist = float('inf')
                match_centroid = None
                for known_centroid in player_ids.keys():
                    dist = np.sqrt((centroid[0] - known_centroid[0])**2 + (centroid[1] - known_centroid[1])**2)
                    if dist < min_dist and dist < 100:  # Threshold distance
                        min_dist = dist
                        match_centroid = known_centroid
                if match_centroid:
                    player_ids[centroid] = player_ids[match_centroid]
                else:
                    new_id = f'player_{len(player_ids)}'
                    player_ids[centroid] = new_id

        # Log tracking (simulate real-time)
        with open('/content/tracking_log.txt', 'a') as f:
            for centroid, pid in player_ids.items():
                f.write(f"Frame {frame_count}, {pid}: {centroid}\n")

        await asyncio.sleep(1.0 / FPS)  # Control frame rate

# Run the async function
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

# Release resources
cap.release()
cv2.destroyAllWindows()

print("Tracking completed and saved to tracking_log.txt")


0: 384x640 1 ball, 16 players, 2 referees, 66.9ms
Speed: 2.2ms preprocess, 66.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 52.9ms
Speed: 2.3ms preprocess, 52.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 40.8ms
Speed: 2.3ms preprocess, 40.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 39.8ms
Speed: 2.4ms preprocess, 39.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 40.8ms
Speed: 3.1ms preprocess, 40.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 39.9ms
Speed: 2.3ms preprocess, 39.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 38.7ms
Speed: 2.4ms preprocess, 38.7ms inference, 1.4ms postprocess per image at sha

In [ ]:
!pip install nest_asyncio

import cv2
import numpy as np
from ultralytics import YOLO
import asyncio
import nest_asyncio
import platform

# Allow nested event loops in Colab
nest_asyncio.apply()

# Load the YOLOv11 model
model_path = '/content/best.pt'
model = YOLO(model_path)

# Video path
video_path = '/content/15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise ValueError("Error opening video file")

# Initialize player tracking
player_ids = {}
frame_count = 0
FPS = 15  # Adjusted for 720p performance
INITIAL_FRAMES = 15  # Adjusted for 1 second at 15 FPS

async def main():
    global frame_count
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        results = model(frame, imgsz=640)  # Reduced size for faster inference

        # Extract bounding boxes
        def get_player_boxes(results):
            boxes = []
            for result in results:
                for box in result.boxes:
                    if int(box.cls) == 0:  # Assuming class 0 is player
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        conf = box.conf[0]
                        boxes.append((x1, y1, x2, y2, conf))
            return boxes

        current_boxes = get_player_boxes(results)

        # Assign IDs in initial frames
        if frame_count <= INITIAL_FRAMES and current_boxes:
            for i, box in enumerate(current_boxes):
                centroid = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
                player_ids[centroid] = f'player_{i}'

        # Match re-entered players
        if frame_count > INITIAL_FRAMES and current_boxes:
            for box in current_boxes:
                centroid = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
                min_dist = float('inf')
                match_centroid = None
                for known_centroid in list(player_ids.keys()):
                    dist = np.sqrt((centroid[0] - known_centroid[0])**2 + (centroid[1] - known_centroid[1])**2)
                    if dist < min_dist and dist < 100:  # Threshold distance
                        min_dist = dist
                        match_centroid = known_centroid
                if match_centroid:
                    player_ids[centroid] = player_ids[match_centroid]
                else:
                    new_id = f'player_{len(player_ids)}'
                    player_ids[centroid] = new_id

        # Log tracking and visualize
        with open('/content/tracking_log.txt', 'a') as f:
            if current_boxes:
                for centroid, pid in player_ids.items():
                    x1, y1, x2, y2, _ = next((b for b in current_boxes if ((b[0] + b[2]) / 2, (b[1] + b[3]) / 2) == centroid), (0, 0, 0, 0, 0))
                    if x1 > 0:  # Only log and draw if box is valid
                        f.write(f"Frame {frame_count}, {pid}: {centroid}\n")
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, pid, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        cv2.imwrite(f'/content/frame_{frame_count}.jpg', frame)

        await asyncio.sleep(1.0 / FPS)  # Control frame rate

# Run the async function
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

# Release resources
cap.release()
cv2.destroyAllWindows()

print("Tracking completed and saved to tracking_log.txt with frame images.")


0: 384x640 1 ball, 16 players, 2 referees, 66.5ms
Speed: 2.1ms preprocess, 66.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 37.3ms
Speed: 2.5ms preprocess, 37.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 37.4ms
Speed: 2.4ms preprocess, 37.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 37.6ms
Speed: 2.1ms preprocess, 37.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 37.2ms
Speed: 2.5ms preprocess, 37.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 37.2ms
Speed: 2.3ms preprocess, 37.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 37.1ms
Speed: 2.9ms preprocess, 37.1ms inference, 1.5ms postprocess per image at sha

# Frame images

In [ ]:
!ffmpeg -framerate 15 -i /content/frame_%d.jpg -c:v libx264 -pix_fmt yuv420p /content/output.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab